In [ ]:
import mysql.connector

**Pulling data to Python to perform matching**

In [ ]:
cnx = mysql.connector.connect(host = '127.0.0.1',database='test',user='root',password='')
jobs_desc = """
SELECT * FROM job_post;
"""
skills = """SELECT * FROM skillset;"""
candidate_skills = """SELECT * FROM user_skillset;"""

def ret_results(q):
    conn = cnx.cursor()
    conn.execute(q)
    ls = []
    for x in conn:
        ls.append(x)
    return ls

jds = ret_results(jobs_desc)
skill_ls = ret_results(skills)
user_skills = ret_results(candidate_skills)

In [ ]:
jds = [x[4] for x in jds]


In [ ]:
jds_id = [x[0] for x in jds]

In [ ]:
skill_ls
skills_dt = {v:k for (k,v) in skill_ls}

In [ ]:
skls = ['Python', 'Tableau', 'Power BI', 'TensorFlow', 'NoSQL', 'PHP', 'Angluar', 'Django', 'Javascript', 'Java', 'SQL', 'MS Word', 'Leadership', 'CSS', 'Fitness', 'AWS', 'Azure','CosmosDB', 'MongoDB', 'SSIS']
len(skls)
skls_st = set(skls)

In [ ]:
from random import sample
skillzzzz = sample(skls, 9)
skillzzzz

**Skills for each job ID**

In [ ]:
import collections
cdt = collections.defaultdict(list)
x = 0
for id in job_ids:
    jd = test[x]
    x += 1
    jd_skills = []
    for word in jd.split():
        if word in skls_st:
            jd_skills.append(word)      
    cdt[id] = jd_skills

cdt

In [ ]:
skls_st

In [ ]:
skill_id_dt = dict()
for ind in range(1,21):
    skill_id_dt[skls[ind - 1]] = ind
skill_id_dt

**Inserting skills for each job**

In [ ]:
st = ''
for job_id, skills in cdt.items():
    for skl in skills:
        qr = ("INSERT INTO job_post_skillset (Job_ID, Skillset_ID) VALUES ({},{})".format(job_id, skill_id_dt[skl]))
        st += qr
        conn = cnx.cursor()
        conn.execute(qr)
        
        
    

## Comparing similarity of jobs with candidate skillset

We use a metric called jaccard similarity, which is basically the proportion of common word between two strings/iterators.

In [ ]:
def get_jaccard_sim(x, y): 
    a = set(x) 
    b = set(y)
    c = a.intersection(b)
    return round(float(len(c)) / (len(a) + len(b) - len(c)),2)

In [ ]:
import collections
user_dt = collections.defaultdict(list)
for (id, skill) in user_skills:  
    user_dt[id].append(skill)

In [ ]:
user_job_match_dt = collections.defaultdict(list)
for user in user_dt.items():
    skills_candidate = user[1]
    for job in cdt.items():
        skills_job = job[1]
        score = get_jaccard_sim(skills_candidate, skills_job)
        user_job_match_dt[user[0]].append((job[0], score))

#We finally sort the jobs for each user based on higest jaccard similarity.
for (x,y) in user_job_match_dt.items():
    temp = sorted(y, key = lambda z: z[1], reverse = True)
    user_job_match_dt[x] = temp
    
